# PostgreSQL — Product Recommendation (SQL)

## Objective
This notebook benchmarks a SQL-based recommendation backend using PostgreSQL.

We measure:
1. Database build time (reset + insert) for increasing volumes
2. Execution time of core recommendation queries

## Project structure
- data_generator.py → generates synthetic rows (low-complexity)
- pgsql_repository.py → schema init, fast reset, bulk insert, query functions


## 1. Environment Setup

We import standard libraries for timing and reporting, plus project modules.

In [1]:
import os, sys
import time
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

from pgsql_repository import (
    create_pg_client, close_pg_client,
    init_schema,
    build_sql,
    similar_by_category, similar_by_similarity,
    customers_also_bought, user_recommendations,
)


## 1. Connection pgsql



In [2]:
pg = create_pg_client()

init_schema(pg, force_recreate=True)


✅ Connected to PostgreSQL successfully.

=== Connected to PostgreSQL (SQL) ===
⚠ Dropping existing benchmark tables (force recreate)...
✔ Old schema dropped.
✔ SQL schema initialized (tables + indexes).


## 3. Benchmark Helpers

We define reusable helper functions to:
- build the graph with timing,
- generate the standard recommendation query set,
- measure query execution time.

These helpers are volume-agnostic and will be reused in each experiment cell.

In [ ]:
import time
from typing import Any, Dict, List

import pandas as pd

from pgsql_repository import (
    build_sql,
    similar_by_category,
    similar_by_similarity,
    customers_also_bought,
    user_recommendations,
)

# NOTE:
# This code assumes you already created a global PostgreSQL connection named `pg`
# using create_pg_client() from pgsql_repository.py, for example:
#   from pgsql_repository import create_pg_client
#   pg = create_pg_client()


# Role:
#   Build (reset + generate + insert) the SQL dataset for a given volume N and measure wall time.
# Inputs:
#   - N (int): dataset size (number of products, minimum 10)
#   - page_size (int): batch size used by execute_values inserts
#   - keep_gremlin_bug_interactions_per_user (bool): if True, keep interactions_per_user=5 to match Gremlin script behavior
# Output:
#   - dict: build statistics returned by build_sql + build_wall_sec + volume
def build_db_with_timing(
    N: int,
    *,
    page_size: int = 5000,
    keep_gremlin_bug_interactions_per_user: bool = True,
) -> Dict[str, Any]:

    N = max(10, int(N))

    t0 = time.perf_counter()
    stats = build_sql(
        pg,
        total_products=N,
        page_size=page_size,
        keep_gremlin_bug_interactions_per_user=bool(keep_gremlin_bug_interactions_per_user),
    )
    wall = time.perf_counter() - t0

    out = stats.copy()
    out["build_wall_sec"] = wall
    out["volume"] = N
    return out


# Role:
#   Run an end-to-end SQL experiment:
#     1) build the SQL dataset for volume N
#     2) run the 4 reference recommendation queries
#     3) return a DataFrame containing timing + number of results
# Inputs:
#   - N (int): dataset size (number of products)
#   - product_id (str): product used for product-based queries
#   - user_id (str): user used for user-based query
#   - page_size (int): insert batch size
#   - keep_gremlin_bug_interactions_per_user (bool): keep Gremlin-like interaction scaling (often always 5)
# Output:
#   - pd.DataFrame: rows of measurements (build phases + query phases)
def run_volume_experiment_sql(
    N: int,
    product_id: str = "p1",
    user_id: str = "u1",
    *,
    page_size: int = 5000,
    keep_gremlin_bug_interactions_per_user: bool = True,
) -> pd.DataFrame:

    rows: List[Dict[str, Any]] = []

    print_banner = lambda title: print(f"\n==============================\n=== {title} ===\n==============================")

    print_banner(f"Build SQL for N={N}")
    stats = build_db_with_timing(
        N,
        page_size=page_size,
        keep_gremlin_bug_interactions_per_user=keep_gremlin_bug_interactions_per_user,
    )

    run_pk = stats["run_pk"]

    # Build phase measurements (these are internal phases measured in build_sql)
    rows += [
        {"volume": N, "run_pk": run_pk, "phase": "build_db", "query": "reset_run",     "time_sec": stats["reset_sec"],       "num_results": 0},
        {"volume": N, "run_pk": run_pk, "phase": "build_db", "query": "generate_data", "time_sec": stats["generate_sec"],    "num_results": stats["products"]},
        {"volume": N, "run_pk": run_pk, "phase": "build_db", "query": "insert_data",   "time_sec": stats["insert_sec"],      "num_results": stats["products"]},
        {"volume": N, "run_pk": run_pk, "phase": "build_db", "query": "build_total",   "time_sec": stats["build_total_sec"], "num_results": stats["products"]},
        {"volume": N, "run_pk": run_pk, "phase": "build_db", "query": "build_wall",    "time_sec": stats["build_wall_sec"],  "num_results": stats["products"]},
    ]

    print(f"→ Running SQL queries for run_pk={run_pk}, product={product_id}, user={user_id}")

    # Query 1: Similar products by category
    r1, t1 = similar_by_category(pg, run_pk, product_id)
    print(f"Similar_by_Category         | {t1:.4f} sec | {len(r1)} rows")
    rows.append({"volume": N, "run_pk": run_pk, "phase": "query", "query": "Similar_by_Category", "time_sec": t1, "num_results": len(r1)})

    # Query 2: Similar products by similarity edges (SIMILAR_TO)
    r2, t2 = similar_by_similarity(pg, run_pk, product_id)
    print(f"Similar_by_SIMILAR_TO       | {t2:.4f} sec | {len(r2)} rows")
    rows.append({"volume": N, "run_pk": run_pk, "phase": "query", "query": "Similar_by_SIMILAR_TO", "time_sec": t2, "num_results": len(r2)})

    # Query 3: Customers also bought
    r3, t3 = customers_also_bought(pg, run_pk, product_id)
    print(f"Customers_Also_Bought       | {t3:.4f} sec | {len(r3)} rows")
    rows.append({"volume": N, "run_pk": run_pk, "phase": "query", "query": "Customers_Also_Bought", "time_sec": t3, "num_results": len(r3)})

    # Query 4: User-based recommendations
    r4, t4 = user_recommendations(pg, run_pk, user_id)
    print(f"User_Recommendations        | {t4:.4f} sec | {len(r4)} rows")
    rows.append({"volume": N, "run_pk": run_pk, "phase": "query", "query": "User_Recommendations", "time_sec": t4, "num_results": len(r4)})

    return pd.DataFrame(rows)


## 4. Experiment A — Small Volume (500 products)

In [4]:
dfs_500 = run_volume_experiment_sql(
    500,
    product_id="p1",
    user_id="u1",
    keep_gremlin_bug_interactions_per_user=True,
)

display(dfs_500)



=== Build SQL for N=500 ===
→ Resetting SQL data for run_pk='bench_N500' ...
✔ Run partition cleared.
→ Bulk inserting rows...
✔ Bulk insert done.
→ Running SQL queries for run_pk=bench_N500, product=p1, user=u1
Similar_by_Category         | 0.0008 sec | 20 rows
Similar_by_SIMILAR_TO       | 0.0005 sec | 3 rows
Customers_Also_Bought       | 0.0096 sec | 1 rows
User_Recommendations        | 4.4659 sec | 4 rows


,volume,run_pk,phase,query,time_sec,num_results
0,500,bench_N500,build_db,reset_run,0.010789,0
1,500,bench_N500,build_db,generate_data,0.004780,500
2,500,bench_N500,build_db,insert_data,0.367583,500
3,500,bench_N500,build_db,build_total,0.383152,500
4,500,bench_N500,build_db,build_wall,0.383475,500
5,500,bench_N500,query,Similar_by_Category,0.000756,20
6,500,bench_N500,query,Similar_by_SIMILAR_TO,0.000456,3
7,500,bench_N500,query,Customers_Also_Bought,0.009573,1
8,500,bench_N500,query,User_Recommendations,4.465854,4


## 5. Experiment B — Medium Volume (1000 products)

In [5]:
dfs_1000 = run_volume_experiment_sql(
    1000,
    product_id="p1",
    user_id="u1",
    keep_gremlin_bug_interactions_per_user=True,
)

display(dfs_1000)



=== Build SQL for N=1000 ===
→ Resetting SQL data for run_pk='bench_N1000' ...
✔ Run partition cleared.
→ Bulk inserting rows...
✔ Bulk insert done.
→ Running SQL queries for run_pk=bench_N1000, product=p1, user=u1
Similar_by_Category         | 0.0010 sec | 20 rows
Similar_by_SIMILAR_TO       | 0.0006 sec | 3 rows
Customers_Also_Bought       | 0.0360 sec | 0 rows
User_Recommendations        | 39.4166 sec | 2 rows


,volume,run_pk,phase,query,time_sec,num_results
0,1000,bench_N1000,build_db,reset_run,0.002375,0
1,1000,bench_N1000,build_db,generate_data,0.008866,1000
2,1000,bench_N1000,build_db,insert_data,0.982891,1000
3,1000,bench_N1000,build_db,build_total,0.994133,1000
4,1000,bench_N1000,build_db,build_wall,0.994351,1000
5,1000,bench_N1000,query,Similar_by_Category,0.001043,20
6,1000,bench_N1000,query,Similar_by_SIMILAR_TO,0.000608,3
7,1000,bench_N1000,query,Customers_Also_Bought,0.036044,0
8,1000,bench_N1000,query,User_Recommendations,39.416563,2


## 6. Experiment C — Large Volume (2000 products)

In [6]:
dfs_2000 = run_volume_experiment_sql(
    2000,
    product_id="p1",
    user_id="u1",
    keep_gremlin_bug_interactions_per_user=True,
)

display(dfs_2000)



=== Build SQL for N=2000 ===
→ Resetting SQL data for run_pk='bench_N2000' ...
✔ Run partition cleared.
→ Bulk inserting rows...
✔ Bulk insert done.
→ Running SQL queries for run_pk=bench_N2000, product=p1, user=u1
Similar_by_Category         | 0.0008 sec | 20 rows
Similar_by_SIMILAR_TO       | 0.0007 sec | 3 rows
Customers_Also_Bought       | 0.1304 sec | 0 rows
User_Recommendations        | 257.4340 sec | 0 rows


,volume,run_pk,phase,query,time_sec,num_results
0,2000,bench_N2000,build_db,reset_run,0.003827,0
1,2000,bench_N2000,build_db,generate_data,0.066774,2000
2,2000,bench_N2000,build_db,insert_data,3.169817,2000
3,2000,bench_N2000,build_db,build_total,3.240417,2000
4,2000,bench_N2000,build_db,build_wall,3.240878,2000
5,2000,bench_N2000,query,Similar_by_Category,0.000766,20
6,2000,bench_N2000,query,Similar_by_SIMILAR_TO,0.000673,3
7,2000,bench_N2000,query,Customers_Also_Bought,0.130351,0
8,2000,bench_N2000,query,User_Recommendations,257.434045,0


## 7. Consolidated Benchmark Report

In [7]:

df_sql_results = pd.concat([dfs_500, dfs_1000, dfs_2000], ignore_index=True)  # ✅ noms cohérents

print("\n✅ Consolidated raw results (SQL)")
display(df_sql_results)

# --- Query-only comparison (pivot) ---
df_sql_queries = df_sql_results[df_sql_results["phase"] == "query"].copy()

pivot_sql_queries = df_sql_queries.pivot_table(
    index="query",
    columns="volume",
    values="time_sec",
    aggfunc="mean"
).sort_index(axis=1)

print("\n✅ SQL query time comparison (seconds)")
display(pivot_sql_queries)

# --- Build breakdown comparison (reset/generate/insert/total/wall) ---
df_sql_build = df_sql_results[df_sql_results["phase"] == "build_db"].copy()

pivot_sql_build = df_sql_build.pivot_table(
    index="query",        # reset_run, generate_data, insert_data, build_total, build_wall
    columns="volume",
    values="time_sec",
    aggfunc="mean"
).sort_index(axis=1)

print("\n✅ SQL build step comparison (seconds)")
display(pivot_sql_build)



✅ Consolidated raw results (SQL)


,volume,run_pk,phase,query,time_sec,num_results
0,500,bench_N500,build_db,reset_run,0.010789,0
1,500,bench_N500,build_db,generate_data,0.004780,500
2,500,bench_N500,build_db,insert_data,0.367583,500
3,500,bench_N500,build_db,build_total,0.383152,500
4,500,bench_N500,build_db,build_wall,0.383475,500
5,500,bench_N500,query,Similar_by_Category,0.000756,20
6,500,bench_N500,query,Similar_by_SIMILAR_TO,0.000456,3
7,500,bench_N500,query,Customers_Also_Bought,0.009573,1
8,500,bench_N500,query,User_Recommendations,4.465854,4
9,1000,bench_N1000,build_db,reset_run,0.002375,0



✅ SQL query time comparison (seconds)


volume,500,1000,2000
query,,,
Customers_Also_Bought,0.009573,0.036044,0.130351
Similar_by_Category,0.000756,0.001043,0.000766
Similar_by_SIMILAR_TO,0.000456,0.000608,0.000673
User_Recommendations,4.465854,39.416563,257.434045



✅ SQL build step comparison (seconds)


volume,500,1000,2000
query,,,
build_total,0.383152,0.994133,3.240417
build_wall,0.383475,0.994351,3.240878
generate_data,0.004780,0.008866,0.066774
insert_data,0.367583,0.982891,3.169817
reset_run,0.010789,0.002375,0.003827
